#Exploring NCAA Data with BigQuery
https://google.qwiklabs.com/focuses/624?parent=catalog

You've learned how to query the NCAA basketball dataset inside of BigQuery.Checkout the GitHub repo [here](https://github.com/google-cloud-ncaa/demos).

**Objectives**

In this lab we will find and query the NCAA dataset using BigQuery.


* Using BigQuery to Query the NCAA Public Dataset

* Writing and executing queries

Provide your credentials to the runtime

In [3]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [4]:
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [5]:
%load_ext google.colab.data_table

What types of basketball play events are there?

In [10]:
%%bigquery --project bigquery-288322
SELECT
  event_type,
  COUNT(*) AS event_count
FROM `bigquery-public-data.ncaa_basketball.mbb_pbp_sr`
GROUP BY 1
ORDER BY event_count DESC;

,event_type,event_count
0,rebound,790490
1,twopointmiss,425422
2,turnover,410166
3,twopointmade,400648
4,freethrowmade,316637
5,assist,293403
6,threepointmiss,291935
7,personalfoul,243613
8,shootingfoul,183541
9,threepointmade,153967


Which 5 games featured the most three point shots made? How accurate were all the attempts?

In [11]:
%%bigquery --project bigquery-288322
#most three points made
SELECT
  scheduled_date,
  name,
  market,
  alias,
  three_points_att,
  three_points_made,
  three_points_pct,
  opp_name,
  opp_market,
  opp_alias,
  opp_three_points_att,
  opp_three_points_made,
  opp_three_points_pct,
  (three_points_made + opp_three_points_made) AS total_threes
FROM `bigquery-public-data.ncaa_basketball.mbb_teams_games_sr`
WHERE season > 2010
ORDER BY total_threes DESC
LIMIT 5;

,scheduled_date,name,market,alias,three_points_att,three_points_made,three_points_pct,opp_name,opp_market,opp_alias,opp_three_points_att,opp_three_points_made,opp_three_points_pct,total_threes
0,2016-11-18,Tigers,Savannah State,SAV,52,24,46.2,Knights,Middle Georgia,MGC,38,16,42.1,40
1,2016-11-18,Knights,Middle Georgia,MGC,38,16,42.1,Tigers,Savannah State,SAV,52,24,46.2,40
2,2016-11-22,Tigers,Savannah State,SAV,45,23,51.1,Wildcats,Fort Valley State,FVSU,39,14,35.9,37
3,2016-11-22,Wildcats,Fort Valley State,FVSU,39,14,35.9,Tigers,Savannah State,SAV,45,23,51.1,37
4,2017-12-10,Skyhawks,Tennessee-Martin,UTM,35,15,42.9,Cougars,Cincinnati Clermont,CINC,51,21,41.2,36


Which 5 basketball venues have the highest seating capacity?

In [12]:
%%bigquery --project bigquery-288322
SELECT
  venue_name, venue_capacity, venue_city, venue_state
FROM `bigquery-public-data.ncaa_basketball.mbb_teams_games_sr`
GROUP BY 1,2,3,4
ORDER BY venue_capacity DESC
LIMIT 5;

,venue_name,venue_capacity,venue_city,venue_state
0,AT&T Stadium,80000,Arlington,TX
1,University of Phoenix Stadium,72220,Glendale,AZ
2,NRG Stadium,71054,Houston,TX
3,Georgia Dome,71000,Atlanta,GA
4,Lucas Oil Stadium,70000,Indianapolis,IN


Which teams played in the highest scoring game since 2010?

In [13]:
%%bigquery --project bigquery-288322
#highest scoring game of all time
SELECT
  scheduled_date,
  name,
  market,
  alias,
  points_game AS team_points,
  opp_name,
  opp_market,
  opp_alias,
  opp_points_game AS opposing_team_points,
  points_game + opp_points_game AS point_total
FROM `bigquery-public-data.ncaa_basketball.mbb_teams_games_sr`
WHERE season > 2010
ORDER BY point_total DESC
LIMIT 5;

,scheduled_date,name,market,alias,team_points,opp_name,opp_market,opp_alias,opposing_team_points,point_total
0,2017-02-10,Terriers,Wofford,WOF,131,Bulldogs,Samford,SAM,127,258
1,2017-02-10,Bulldogs,Samford,SAM,127,Terriers,Wofford,WOF,131,258
2,2017-02-04,Vikings,Portland State,PRST,124,Eagles,Eastern Washington,EWU,130,254
3,2017-02-04,Eagles,Eastern Washington,EWU,130,Vikings,Portland State,PRST,124,254
4,2013-11-14,Pioneers,Sacred Heart,SHU,118,Crusaders,Holy Cross,HC,122,240


Since 2015, what was the biggest difference in final score for a National Championship?

In [14]:
%%bigquery --project bigquery-288322
#biggest point difference in a championship game
SELECT
  scheduled_date,
  name,
  market,
  alias,
  points_game AS team_points,
  opp_name,
  opp_market,
  opp_alias,
  opp_points_game AS opposing_team_points,
  ABS(points_game - opp_points_game) AS point_difference
FROM `bigquery-public-data.ncaa_basketball.mbb_teams_games_sr`
WHERE season > 2015 AND tournament_type = 'National Championship'
ORDER BY point_difference DESC
LIMIT 5;

,scheduled_date,name,market,alias,team_points,opp_name,opp_market,opp_alias,opposing_team_points,point_difference
0,2018-04-03,Wildcats,Villanova,VILL,79,Wolverines,Michigan,MICH,62,17
1,2018-04-03,Wolverines,Michigan,MICH,62,Wildcats,Villanova,VILL,79,17
2,2017-04-04,Bulldogs,Gonzaga,GONZ,65,Tar Heels,North Carolina,UNC,71,6
3,2017-04-04,Tar Heels,North Carolina,UNC,71,Bulldogs,Gonzaga,GONZ,65,6
